# I. Data Loading and Preprocessing

Data source from `Dreaddit: A Reddit Dataset for Stress Analysis in Social Media`.

Following tasks are undertaken:
* Columns Selection
* Feature Transformation
* Handling Missing Values
* Column Encoding

## Categorical Data Encoding, Feature Selection and Training...

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline
%load_ext lab_black
from sklearn.model_selection import train_test_split
import warnings

warnings.filterwarnings("ignore")

# Data import
dreaddit_train_df = pd.read_csv("../../data/dreaddit/dreaddit-train.csv")
dreaddit_test_df = pd.read_csv("../../data/dreaddit/dreaddit-test.csv")

In [2]:
def plot_graphs(history, metric):
    plt.plot(history.history[metric])
    plt.plot(history.history["val_" + metric], "")
    plt.xlabel("Epochs")
    plt.ylabel(metric)
    plt.legend([metric, "val_" + metric])

In [3]:
dreaddit_train_df.head()

,subreddit,post_id,sentence_range,text,id,label,confidence,social_timestamp,social_karma,syntax_ari,...,lex_dal_min_pleasantness,lex_dal_min_activation,lex_dal_min_imagery,lex_dal_avg_activation,lex_dal_avg_imagery,lex_dal_avg_pleasantness,social_upvote_ratio,social_num_comments,syntax_fk_grade,sentiment
0,ptsd,8601tu,"(15, 20)","He said he had not felt that way before, sugge...",33181,1,0.8,1521614353,5,1.806818,...,1.000,1.1250,1.0,1.77000,1.52211,1.89556,0.86,1,3.253573,-0.002742
1,assistance,8lbrx9,"(0, 5)","Hey there r/assistance, Not sure if this is th...",2606,0,1.0,1527009817,4,9.429737,...,1.125,1.0000,1.0,1.69586,1.62045,1.88919,0.65,2,8.828316,0.292857
2,ptsd,9ch1zh,"(15, 20)",My mom then hit me with the newspaper and it s...,38816,1,0.8,1535935605,2,7.769821,...,1.000,1.1429,1.0,1.83088,1.58108,1.85828,0.67,0,7.841667,0.011894
3,relationships,7rorpp,"[5, 10]","until i met my new boyfriend, he is amazing, h...",239,1,0.6,1516429555,0,2.667798,...,1.000,1.1250,1.0,1.75356,1.52114,1.98848,0.50,5,4.104027,0.141671
4,survivorsofabuse,9p2gbc,"[0, 5]",October is Domestic Violence Awareness Month a...,1421,1,0.8,1539809005,24,7.554238,...,1.000,1.1250,1.0,1.77644,1.64872,1.81456,1.00,1,7.910952,-0.204167


In [4]:
print(dreaddit_train_df.text[5])

I think he doesn't want to put in the effort for the relationship to work (and we're both so difficult that we have to work on our relationships, doesn't matter with whom) but he can't be without me either. What should I do? I'm afraid this is gonna happen over and over again, because I'm always forgiving him at some point. Am I being strung along? TL;DR: Boyfriend [28,M] broke up with me [23,F] after on-off for 1.5 years, I thought we just got it together and am devastated...don't know what to do, want to keep fighting but should I?


In [5]:
cols = dreaddit_train_df.columns

In [6]:
new_cols = [
    "text",
    "label",
]

In [7]:
train_df = dreaddit_train_df[new_cols]
train_df.head()

,text,label
0,"He said he had not felt that way before, sugge...",1
1,"Hey there r/assistance, Not sure if this is th...",0
2,My mom then hit me with the newspaper and it s...,1
3,"until i met my new boyfriend, he is amazing, h...",1
4,October is Domestic Violence Awareness Month a...,1


In [8]:
test_cols = dreaddit_test_df.columns
dreaddit_test_df.info()
test_df = dreaddit_test_df[new_cols]
test_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 715 entries, 0 to 714
Columns: 116 entries, id to sentiment
dtypes: float64(107), int64(5), object(4)
memory usage: 648.1+ KB


,text,label
0,"Its like that, if you want or not.“ ME: I have...",0
1,I man the front desk and my title is HR Custom...,0
2,We'd be saving so much money with this new hou...,1
3,"My ex used to shoot back with ""Do you want me ...",1
4,I haven’t said anything to him yet because I’m...,0


In [9]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2838 entries, 0 to 2837
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    2838 non-null   object
 1   label   2838 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 44.5+ KB


In [10]:
# train_df.subreddit.value_counts()

In [11]:
train_df.label.value_counts()

1    1488
0    1350
Name: label, dtype: int64

# III. TF Feature Columns to Classify Structure Data

In [12]:
import os
import tensorflow as tf
from tensorflow import feature_column
from tensorflow.keras import layers
from tensorflow import keras

import numpy as np

## Create an input pipeline using tf.data

In [13]:
# Convert to tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe):
    dataframe = dataframe.copy()
    labels = dataframe.pop("label")
    ds = tf.data.Dataset.from_tensor_slices((dataframe, labels))
    return ds

In [14]:
train_ds = df_to_dataset(train_df)
test_ds = df_to_dataset(test_df)

In [15]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

In [16]:
train_dataset = (
    train_ds.shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE)
    .prefetch(tf.data.experimental.AUTOTUNE)
)
test_dataset = test_ds.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

In [17]:
train_dataset.element_spec

(TensorSpec(shape=(None, 1), dtype=tf.string, name=None),
 TensorSpec(shape=(None,), dtype=tf.int64, name=None))

In [18]:
from sklearn.model_selection import train_test_split

In [19]:
VOCAB_SIZE = 1000
encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=VOCAB_SIZE
)
encoder.adapt(train_ds.map(lambda text, label: text))

In [20]:
vocab = np.array(encoder.get_vocabulary())
vocab[:20]

array(['', '[UNK]', 'i', 'to', 'and', 'the', 'a', 'my', 'of', 'me',
       'that', 'in', 'it', 'for', 'was', 'is', 'but', 'have', 'with',
       'this'], dtype='<U13')

In [21]:
for example, label in train_dataset.take(1):
    print("texts: ", example.numpy()[:1])
    print()
    print("labels: ", label.numpy()[:1])

texts:  [[b'I never personally did anything beyond a "show me yours and I\'ll show you mine" type of thing where another kid and I would see how we "measured up", but it felt like those experiences had heavily sexual undertones for me - I don\'t know if that was me figuring things out or if my mind went there from what happened to me cousin. When I was about 13 I remember thinking to myself that it was crazy that so many of my peers knew what they wanted - I was just really, really confused. I said to myself "I don\'t know what I like or which team I go, and I bet that\'s because of what happened between me and my cousin. I\'ll figure it out when I\'m older" and that was it. I did what a lot of other kids my age were doing - I hopped online and watched lots of porn.']]

labels:  [0]


In [22]:
encoded_example = encoder(example)[:1].numpy()
encoded_example

array([[  2,  91,   1, 123, 135, 992,   6, 587,   9,   1,   4, 236, 587,
         25, 676, 610,   8, 216, 115, 225, 616,   4,   2,  54, 119,  61,
         34,   1,  39,  16,  12, 189,  32, 273, 573,  42,   1, 547,   1,
         13,   9,   2,  55,  46,  35,  10,  14,   9,   1,  85,  36,  31,
         35,   7, 310, 163,  80,  50,  41, 185,   3,   9,   1,  44,   2,
         14,  30,   1,   2, 304, 271,   3,  98,  10,  12,  14, 598,  10,
         22, 230,   8,   7,   1, 381,  41,  58, 199,   2,  14,  29,  59,
         59, 955,   2, 127,   3,  98,   2,  55,  46,  41,   2,  32,  31,
        112,   1,   2,  73,   4,   2,   1, 277,  43,   8,  41, 185, 391,
          9,   4,   7,   1, 236, 619,  12,  36,  44,  26, 549,   4,  10,
         14,  12,   2, 123,  41,   6, 149,   8, 103, 430,   7, 480,  92,
        182,   2,   1, 541,   4,   1,   1,   8,   1,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0

In [23]:
model = tf.keras.Sequential(
    [
        encoder,
        tf.keras.layers.Embedding(len(encoder.get_vocabulary()), 64, mask_zero=True),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1),
    ]
)

In [24]:
sample_text = "I have been experiencing a depressing mood and my stress levels are rising tremendously. This should indicate something"
predictions = model.predict(np.array([sample_text]))
print(predictions[0])

[0.00210559]


In [25]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, save_weights_only=True, verbose=1
)
logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)


In [26]:
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(1e-4),
    metrics=["accuracy"],
)

In [27]:
history = model.fit(
    train_dataset,
    epochs=10,
    callbacks=[cp_callback, tensorboard_callback],
    validation_data=test_dataset,
    validation_steps=30,
)

Epoch 1/10
45/45 [==============================] - ETA: 0s - loss: 0.6924 - accuracy: 0.4757WARNING:tensorflow:Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 30 batches). You may need to use the repeat() function when building your dataset.

Epoch 00001: saving model to training_1/cp.ckpt
45/45 [==============================] - 50s 1s/step - loss: 0.6924 - accuracy: 0.4757 - val_loss: 0.6920 - val_accuracy: 0.4839
Epoch 2/10
45/45 [==============================] - ETA: 0s - loss: 0.6899 - accuracy: 0.4757
Epoch 00002: saving model to training_1/cp.ckpt
45/45 [==============================] - 36s 798ms/step - loss: 0.6899 - accuracy: 0.4757
Epoch 3/10
45/45 [==============================] - ETA: 0s - loss: 0.6787 - accuracy: 0.4838
Epoch 00003: saving model to training_1/cp.ckpt
45/45 [==============================] - 36s 796ms/step - loss: 0.6787 - accuracy: 0.4838

In [28]:
# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save("saved_model/red_model")

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: saved_model/red_model/assets


In [29]:
test_loss, test_acc = model.evaluate(test_dataset)

print("Test Loss: {}".format(test_loss))
print("Test Accuracy: {}".format(test_acc))

12/12 [==============================] - 3s 259ms/step - loss: 0.5625 - accuracy: 0.7175
Test Loss: 0.5625007152557373
Test Accuracy: 0.7174825072288513


In [30]:
# Predict on a sample to test accuract and padding.

sample_text = "I have been experiencing a depressing mood and my stress levels are rising tremendously. This should indicate something."
predictions = model.predict(np.array([sample_text]))
print(predictions)

[[-0.84668046]]


In [31]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard

%tensorboard --logdir logs/scalars

ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 4709.